#### origin
silver table

In [0]:

CATALOG_NAME = "databricks_hackathon"
SCHEMA_NAME_ORIG = "silver"
TABLE_NAME_1 = "capacitance"
TABLE_NAME_2 = "timeseries"

TABLE_NAME_OUT = "ML_Ready_V1"
SCHEMA_NAME_DEST="gold"
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME_DEST}")

capa_silver = spark.read.table(f"{CATALOG_NAME}.{SCHEMA_NAME_ORIG}.{TABLE_NAME_1}")
time_silver = spark.read.table(f"{CATALOG_NAME}.{SCHEMA_NAME_ORIG}.{TABLE_NAME_2}")

In [0]:
# create views
capa_silver.createOrReplaceTempView("capa_temp_view")
time_silver.createOrReplaceTempView("time_temp_view")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window


def forwardfill_null_values(df_filtered):
    # Apply forward fill one column at a time
    window_spec = Window.partitionBy("batch_id").orderBy("Timestamp").rowsBetween(Window.unboundedPreceding, 0)

    current_df = df_filtered
    for c in df_filtered.columns:
        if c not in ["Timestamp", "batch_id"]:
            all_cols = []
            for col_name in current_df.columns:
                if col_name == c:
                    all_cols.append(F.last(F.col(col_name), ignorenulls=True).over(window_spec).alias(col_name))
                    print(f"filled col {col_name}")
                else:
                    all_cols.append(F.col(col_name))
            
            current_df = current_df.select(*all_cols)
    current_df = current_df.fillna(0)
    return current_df

#### select columns to join based on EDA

In [0]:
join_query = """
    SELECT
        c.*,
        t.Timestamp,
        t.m_do_percent,
        t.m_air_ml_min,
        t.co_stirrer_percent
    FROM
        capa_temp_view c
    INNER JOIN
        time_temp_view t
    ON
        c.batch_id = t.batch_id AND c.Time_Stamp = t.Timestamp
"""

# Execute the join query
joined_df = spark.sql(join_query)


#final_df = forwardfill_null_values(joined_df)
joined_df.show(5)

In [0]:

joined_df.write \
            .format("delta") \
            .option("mergeSchema", "true") \
            .mode("overwrite") \
            .partitionBy("project", "batch_id") \
            .saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME_ORIG}.{TABLE_NAME_OUT}")

In [0]:
joined_df_2 = forwardfill_null_values(joined_df)

joined_df.write \
            .format("delta") \
            .option("mergeSchema", "true") \
            .mode("overwrite") \
            .partitionBy("project", "batch_id") \
            .saveAsTable(f"{CATALOG_NAME}.{SCHEMA_NAME_ORIG}.ML_Ready_V2")